In [1]:
pip install duckdb --upgrade

Note: you may need to restart the kernel to use updated packages.


In [21]:
import wrds
import duckdb
import pandas as pd

In [22]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [23]:
db.list_libraries()

['aha_sample',
 'ahasamp',
 'audit',
 'audit_acct_os',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'bank_premium_samp',
 'banksamp',
 'block',
 'block_all',
 'boardex',
 'boardex_eur',
 'boardex_na',
 'boardex_row',
 'boardex_trial',
 'boardex_uk',
 'boardsmp',
 'bvd',
 'bvd_ama_large',
 'bvd_ama_medium',
 'bvd_ama_small',
 'bvd_ama_verylarge',
 'bvd_amadeus_trial',
 'bvd_bvdbankf',
 'bvd_bvdbankf_trial',
 'bvd_orbis_large',
 'bvd_orbis_medium',
 'bvd_orbis_small',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'ciq',
 'ciq_common',
 'ciq_keydev',
 'ciq_ratings',
 'ciq_transcripts',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_filings',
 'comp

In [24]:
db.list_tables(library="crsp_q_stock")

['dse',
 'dseall',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',
 'dsf_v2',
 'dsfhdr',
 'dsi',
 'dsiy',
 'inddlyseriesdata',
 'indfamilyinfohdr',
 'indmthseriesdata',
 'indseriesinfohdr',
 'metacalendarperiod',
 'metacolumncoverage',
 'metacolumninfo',
 'metaexchangecalendar',
 'metafileinfo',
 'metaflagcoverage',
 'metaflaginfo',
 'metaflagtype',
 'metaiteminfo',
 'metasiztociz',
 'mse',
 'mseall',
 'msedelist',
 'msedist',
 'mseexchdates',
 'msenames',
 'msenasdin',
 'mseshares',
 'msf',
 'msf_v2',
 'msfhdr',
 'msi',
 'msiy',
 'saz_agg_ann',
 'saz_agg_ann_legacy',
 'saz_agg_mth',
 'saz_agg_mth_legacy',
 'saz_agg_qtr',
 'saz_agg_qtr_legacy',
 'saz_del',
 'saz_del_legacy',
 'saz_dind',
 'saz_dind_legacy',
 'saz_dis',
 'saz_dis_legacy',
 'saz_dp_dly',
 'saz_dp_dly_legacy',
 'saz_ds_dly',
 'saz_ds_dly_legacy',
 'saz_hdr',
 'saz_hdr_legacy',
 'saz_indhdr',
 'saz_indhdr_legacy',
 'saz_mdel',
 'saz_mdel_legacy',
 'saz_mind',
 'saz_mind_legacy',


In [25]:
db.describe_table(library="crsp_q_stock", table="metacolumninfo")

Approximately 567 rows in crsp_q_stock.metacolumninfo.


,name,nullable,type,comment
0,filename,True,VARCHAR(32),File Name
1,colposition,True,SMALLINT,Column Position
2,itemname,True,VARCHAR(32),Item Name
3,colnullflg,True,VARCHAR(1),Column Null Flag
4,colsqldatatype,True,VARCHAR(16),Column SQL Data Type
5,colsasdatatype,True,VARCHAR(4),Column SAS Data Type
6,colsasdisplayformat,True,VARCHAR(16),Column SAS Display Format
7,colsasexportformat,True,VARCHAR(16),Column SAS Export Format
8,colasciimaxfieldsize,True,SMALLINT,Column ASCII Maximum Field Size
9,colrdatatype,True,VARCHAR(10),Column R Data Type


In [26]:
db.describe_table(library="crsp_q_stock", table="metacolumninfo")['type']

Approximately 567 rows in crsp_q_stock.metacolumninfo.


0     VARCHAR(32)
1        SMALLINT
2     VARCHAR(32)
3      VARCHAR(1)
4     VARCHAR(16)
5      VARCHAR(4)
6     VARCHAR(16)
7     VARCHAR(16)
8        SMALLINT
9     VARCHAR(10)
10     VARCHAR(1)
11        INTEGER
12        INTEGER
13       SMALLINT
Name: type, dtype: object

In [27]:
unique_types = db.describe_table(library="crsp_q_stock", table="metacolumninfo")['type'].astype(str).drop_duplicates()

print(unique_types)

Approximately 567 rows in crsp_q_stock.metacolumninfo.
0     VARCHAR(32)
1        SMALLINT
3      VARCHAR(1)
4     VARCHAR(16)
5      VARCHAR(4)
9     VARCHAR(10)
11        INTEGER
Name: type, dtype: object


In [29]:
# Create DuckDB connection
duck_conn = duckdb.connect('wrds_data.db')


# List of tables to process
tables = [
    'metacalendarperiod', 'metacolumncoverage', 'metacolumninfo', 
    'metaexchangecalendar', 'metafileinfo', 'metaflagcoverage', 
    'metaflaginfo', 'metaflagtype', 'metaiteminfo', 'metasiztociz'
]

for table_name in tables:
    print(f"\nProcessing table: {table_name}")
    
    # Get schema information
    schema_info = db.describe_table(library="crsp_q_stock", table=table_name)

    # Drop existing table if it exists
    duck_conn.execute(f"DROP TABLE IF EXISTS {table_name}")

    # Create table with exact schema mapping
    create_table_sql = f"CREATE TABLE {table_name} ("
    columns = []
    for _, row in schema_info.iterrows():
        col_name = row['name']
        wrds_type = str(row['type']).upper()
        
        if wrds_type == 'DATE':
            duck_type = 'DATE'
        elif wrds_type.startswith('VARCHAR'):
            duck_type = wrds_type
        elif wrds_type == 'SMALLINT':
            duck_type = 'SMALLINT'
        elif wrds_type == 'INTEGER':
            duck_type = 'INTEGER'
        elif wrds_type.startswith('NUMERIC'):
            duck_type = wrds_type.replace('NUMERIC', 'DECIMAL')
        else:
            duck_type = 'VARCHAR'
            
        columns.append(f"{col_name} {duck_type}")

    create_table_sql += ", ".join(columns) + ")"
    duck_conn.execute(create_table_sql)

    # Get row count first (for progress tracking)
    row_count = db.raw_sql(f"SELECT COUNT(*) FROM crsp_q_stock.{table_name}").iloc[0,0]
    print(f"Total rows to process: {row_count}")

    # Process in chunks
    chunksize = 100000  # Adjust based on table size
    offset = 0

    while True:
        query = f"""
            select * 
            from crsp_q_stock.{table_name} 
            LIMIT {chunksize} 
            OFFSET {offset}
        """
        
        try:
            chunk = db.raw_sql(query)
            
            if len(chunk) == 0:
                break
            
            # Handle date columns properly
            date_cols = schema_info[schema_info['type'].astype(str).str.upper() == 'DATE']['name'].tolist()
            for col in date_cols:
                if col in chunk.columns:
                    def convert_date(x):
                        if pd.isna(x):
                            return None
                        elif str(x) == '9999-12-31':
                            return 'infinity'
                        try:
                            return pd.to_datetime(x).strftime('%Y-%m-%d')
                        except:
                            return None
                    
                    chunk[col] = chunk[col].apply(convert_date)
            
            # Insert the chunk into DuckDB
            duck_conn.register('temp_chunk', chunk)
            duck_conn.sql(f'INSERT INTO {table_name} SELECT * FROM temp_chunk')
            
            offset += chunksize
            print(f"Processed {min(offset, row_count)} rows out of {row_count}")
            
        except Exception as e:
            print(f"Error processing {table_name} at offset {offset}: {str(e)}")
            raise

    print(f"Completed processing table: {table_name}")





Processing table: metacalendarperiod
Approximately 28473 rows in crsp_q_stock.metacalendarperiod.
Total rows to process: 28473
Processed 28473 rows out of 28473
Completed processing table: metacalendarperiod

Processing table: metacolumncoverage
Approximately 118 rows in crsp_q_stock.metacolumncoverage.
Total rows to process: 118
Processed 118 rows out of 118
Completed processing table: metacolumncoverage

Processing table: metacolumninfo
Approximately 567 rows in crsp_q_stock.metacolumninfo.
Total rows to process: 567
Processed 567 rows out of 567
Completed processing table: metacolumninfo

Processing table: metaexchangecalendar
Approximately 36069 rows in crsp_q_stock.metaexchangecalendar.
Total rows to process: 36069
Processed 36069 rows out of 36069
Completed processing table: metaexchangecalendar

Processing table: metafileinfo
Approximately 32 rows in crsp_q_stock.metafileinfo.
Total rows to process: 32
Processed 32 rows out of 32
Completed processing table: metafileinfo

Proces

In [30]:
duck_conn.close()
db.close()